In [2]:
import requests
from bs4 import BeautifulSoup

In [3]:
stories = ['null']

def get_soup(url):
	response = requests.get(url)
	soup = BeautifulSoup(response.text, "xml")	
	return soup


def get_headlines():
	scrape_url = "http://feeds.bbci.co.uk/news/rss.xml"
	current_item = 0
	soup = get_soup(scrape_url)
	articles = soup.find_all('item')
	res = []
	while current_item < len(articles):
		article_headline = articles[current_item].title.text
		article_description = ["".join(article_part + " " for article_part in articles[current_item].text.split("\n")[:-5])][0] 
		res.append(f"{article_headline} {article_description}")
		current_item += 1
	return res

res = dict()
for i,x in enumerate(get_headlines()):
	res[i]=x

In [4]:
# Use a pipeline as a high-level helper
from transformers import pipeline

pipe = pipeline("sentiment-analysis", model="Lukiccc/my_awesome_model")

2024-08-22 20:03:49.280220: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-08-22 20:03:49.297300: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-08-22 20:03:49.316467: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-08-22 20:03:49.322199: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-08-22 20:03:49.338964: I tensorflow/core/platform/cpu_feature_guar

In [14]:

from datetime import datetime

res = dict()
for pred, text in zip(pipeline("sentiment-analysis", model="Lukiccc/my_awesome_model")(list(get_headlines())), get_headlines()):
    res[text] = pred

print(res)
starting_item = 0
today = datetime.today()

print(res.values())

res_s = ''
for i, (key, value) in enumerate(res.items()):
    text = key
    label = value['label']
    if i == starting_item:
        res_s += f'({today}, {i}, {text}, {label})'
    else:
        res_s += f',({today}, {i}, {text}, {label})'

print(res_s)

{"Body of UK tech tycoon recovered but daughter still missing  Body of UK tech tycoon recovered but daughter still missing UK tycoon Mike Lynch's body is recovered from the wreck of the yacht Bayesian, but his daughter Hannah is still missing. ": {'label': 'world', 'score': 0.50132817029953}, 'Bayesian sinking: The key questions for investigators  Bayesian sinking: The key questions for investigators Experts tell the BBC that investigators will be asking if a freak weather event sank the Bayesian. ': {'label': 'science', 'score': 0.9529939293861389}, 'BBC sacks One Show presenter Jermaine Jenas after complaints  BBC sacks One Show presenter Jermaine Jenas after complaints The former footballer is taken off air at Match of the Day and the One Show. ': {'label': 'sport', 'score': 0.9843829274177551}, 'Rattled Trump lashes out as DNC attacks throw him off message  Rattled Trump lashes out as DNC attacks throw him off message The biggest concern for aides is keeping him focused on the issu

In [50]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForSequenceClassification

tokenizer = AutoTokenizer.from_pretrained("Lukiccc/my_awesome_model")
model = AutoModelForSequenceClassification.from_pretrained("Lukiccc/my_awesome_model")